In [3]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value ### fro maximization problem use Lpmaximize 
import pandas as pd

# Define data
ports = ['Vancouver', 'Boston', 'Miami', 'San Francisco']
refineries = ['Denver', 'Atlanta', 'Pittsburgh']
outlets = ['Dallas', 'Phoenix', 'Portland', 'Montreal', 'Orlando']

supply = {'Vancouver': 1000, 'Boston': 800, 'Miami': 800, 'San Francisco': 700}
demand = {'Dallas': 900, 'Phoenix': 800, 'Portland': 600, 'Montreal': 500, 'Orlando': 500}

port_to_refinery_costs = {
    ('Vancouver', 'Denver'): 4, ('Vancouver', 'Atlanta'): 13, ('Vancouver', 'Pittsburgh'): 9,
    ('Boston', 'Denver'): 8, ('Boston', 'Atlanta'): 8, ('Boston', 'Pittsburgh'): 5,
    ('Miami', 'Denver'): 12, ('Miami', 'Atlanta'): 2, ('Miami', 'Pittsburgh'): 9,
    ('San Francisco', 'Denver'): 11, ('San Francisco', 'Atlanta'): 11, ('San Francisco', 'Pittsburgh'): 12
}

refinery_to_outlet_costs = {
    ('Denver', 'Dallas'): 28, ('Denver', 'Phoenix'): 26, ('Denver', 'Portland'): 12, ('Denver', 'Montreal'): 30, ('Denver', 'Orlando'): 30,
    ('Atlanta', 'Dallas'): 10, ('Atlanta', 'Phoenix'): 22, ('Atlanta', 'Portland'): 23, ('Atlanta', 'Montreal'): 29, ('Atlanta', 'Orlando'): 8,
    ('Pittsburgh', 'Dallas'): 18, ('Pittsburgh', 'Phoenix'): 21, ('Pittsburgh', 'Portland'): 23, ('Pittsburgh', 'Montreal'): 18, ('Pittsburgh', 'Orlando'): 21
}

# Define the problem
prob = LpProblem("Two_Stage_Transportation", LpMinimize)

# Variables for waht goes into refineries must equal shipment to outlets 
port_to_refinery_vars = {
    (p, r): LpVariable(f"{p}_to_{r}", lowBound=0)
    for p in ports for r in refineries
}
refinery_to_outlet_vars = {
    (r, o): LpVariable(f"{r}_to_{o}", lowBound=0)
    for r in refineries for o in outlets
}

# Objective function
prob += lpSum(
    port_to_refinery_costs[(p, r)] * port_to_refinery_vars[(p, r)]
    for p in ports for r in refineries
) + lpSum(
    refinery_to_outlet_costs[(r, o)] * refinery_to_outlet_vars[(r, o)]
    for r in refineries for o in outlets
)

# Constraints for ports, refineries and outlets 
for p in ports:
    prob += lpSum(port_to_refinery_vars[(p, r)] for r in refineries) == supply[p]

for o in outlets:
    prob += lpSum(refinery_to_outlet_vars[(r, o)] for r in refineries) == demand[o]

for r in refineries:
    prob += (
        lpSum(port_to_refinery_vars[(p, r)] for p in ports)
        == lpSum(refinery_to_outlet_vars[(r, o)] for o in outlets)
    )

# Solve the problem with solver 
prob.solve()

# Results organized with pandas as a data frame 
port_to_refinery_df = pd.DataFrame([
    {"Port": p, "Refinery": r, "Shipment": value(port_to_refinery_vars[(p, r)])}
    for p in ports for r in refineries
])

refinery_to_outlet_df = pd.DataFrame([
    {"Refinery": r, "Outlet": o, "Shipment": value(refinery_to_outlet_vars[(r, o)])}
    for r in refineries for o in outlets
])

total_cost = value(prob.objective ##objective function

# Output results
print("Port to Refinery Shipments:")
print(port_to_refinery_df)
print("\nRefinery to Outlet Shipments:")
print(refinery_to_outlet_df)
print("\nTotal Transportation Cost:")
print(total_cost)


Port to Refinery Shipments:
             Port    Refinery  Shipment
0       Vancouver      Denver     600.0
1       Vancouver     Atlanta       0.0
2       Vancouver  Pittsburgh     400.0
3          Boston      Denver       0.0
4          Boston     Atlanta       0.0
5          Boston  Pittsburgh     800.0
6           Miami      Denver       0.0
7           Miami     Atlanta     800.0
8           Miami  Pittsburgh       0.0
9   San Francisco      Denver       0.0
10  San Francisco     Atlanta     700.0
11  San Francisco  Pittsburgh       0.0

Refinery to Outlet Shipments:
      Refinery    Outlet  Shipment
0       Denver    Dallas       0.0
1       Denver   Phoenix       0.0
2       Denver  Portland     600.0
3       Denver  Montreal       0.0
4       Denver   Orlando       0.0
5      Atlanta    Dallas     900.0
6      Atlanta   Phoenix     100.0
7      Atlanta  Portland       0.0
8      Atlanta  Montreal       0.0
9      Atlanta   Orlando     500.0
10  Pittsburgh    Dallas       0.0
1